In [1]:
%pip install qiskit

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade pip
!pip install "iqm-client[qiskit]"
!pip install qrisp[iqm]
!pip install numpy==2.0
!pip install matplotlib
!pip install pylatexenc


  Using cached numpy-2.3.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.3.5-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
  Using cached numpy-2.0.0-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
Using cached numpy-2.0.0-cp311-cp311-macosx_14_0_arm64.whl (5.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numpy-typing-compat 20251206.2.3 requires numpy<2.4,>=2.3, but you have numpy 2.0.0 which is incompatible.


In [3]:
from qiskit import QuantumCircuit

def create_ghz_circuit(qubit_count):
    qc = QuantumCircuit(qubit_count)

    # Build the GHZ circuit
    qc.h(0)
    for qb in range(1, qubit_count):
        qc.cx(0, qb)

    return qc


In [6]:
def basis_measurement(qc: QuantumCircuit, basis: str):
    """Adds measurement in the specified basis to the circuit."""
    n = qc.num_qubits
    meas_circ = QuantumCircuit(n, n)
    meas_circ.compose(qc, range(n), inplace=True)

    if basis == 'X':
        for qubit in range(qc.num_qubits):
            meas_circ.h(qubit)
    elif basis == 'Y':
        for qubit in range(qc.num_qubits):
            meas_circ.sdg(qubit)
            meas_circ.h(qubit)
    elif basis == 'Z':
        pass
    else:
        raise ValueError("Basis must be 'X', 'Y', or 'Z'")
    
    # For 'Z' basis, no additional gates are needed
    meas_circ.measure(range(n), range(n))
    meas_circ.metadata = {'basis': basis}
    return meas_circ

N = 6
ghz6 = create_ghz_circuit(N)

circ_X = basis_measurement(ghz6, 'X')
circ_Y = basis_measurement(ghz6, 'Y')
circ_Z = basis_measurement(ghz6, 'Z')

circuits = [circ_X, circ_Y, circ_Z]

print("Circuits prepared for measurement in X, Y, and Z bases.")
print(circ_X.draw(output='text'))
print(circ_Y.draw(output='text'))
print(circ_Z.draw(output='text'))

Circuits prepared for measurement in X, Y, and Z bases.
     ┌───┐                                         ┌───┐        ┌─┐   
q_0: ┤ H ├──■────■─────────■───────■────────────■──┤ H ├────────┤M├───
     └───┘┌─┴─┐  │  ┌───┐  │  ┌─┐  │            │  └───┘        └╥┘   
q_1: ─────┤ X ├──┼──┤ H ├──┼──┤M├──┼────────────┼────────────────╫────
          └───┘┌─┴─┐├───┤  │  └╥┘  │       ┌─┐  │                ║    
q_2: ──────────┤ X ├┤ H ├──┼───╫───┼───────┤M├──┼────────────────╫────
               └───┘└───┘┌─┴─┐ ║   │  ┌───┐└╥┘  │   ┌─┐          ║    
q_3: ────────────────────┤ X ├─╫───┼──┤ H ├─╫───┼───┤M├──────────╫────
                         └───┘ ║ ┌─┴─┐├───┤ ║   │   └╥┘      ┌─┐ ║    
q_4: ──────────────────────────╫─┤ X ├┤ H ├─╫───┼────╫───────┤M├─╫────
                               ║ └───┘└───┘ ║ ┌─┴─┐  ║  ┌───┐└╥┘ ║ ┌─┐
q_5: ──────────────────────────╫────────────╫─┤ X ├──╫──┤ H ├─╫──╫─┤M├
                               ║            ║ └───┘  ║  └───┘ ║  ║ └╥┘
c: 6/════════════════

In [ ]:
from iqm.qiskit_iqm import IQMProvider

SHOTS = 1024 

provider = IQMProvider("https://resonance.meetiqm.com",
                       quantum_computer="garnet",
                       token="aJZeUpSK3smE5rBKnZmcOGrmDmBwWC68k0Z3AEQ+gEkBnBR9Gkxz4YDOlNha+WKz")
backend = provider.get_backend()

qc_transpiled = transpile(qc,backend)
job = backend.run(qc_transpiled, SHOTS=SHOTS)

result=job.result()

counts_X = result.get_counts(0)  # first circuit
counts_Y = result.get_counts(1)  # second circuit
counts_Z = result.get_counts(2)  # third circuit


print("Counts (X basis):", counts_X)
print("Counts (Y basis):", counts_Y)
print("Counts (Z basis):", counts_Z)

print(result.get_counts())

TypeError: expected string or bytes-like object, got 'NoneType'

In [ ]:
def exp_XN_or_YN_from_counts(counts, n_qubits, shots):
    exp_value = 0
    for bitstring, count in counts.items():
        parity = sum(int(bit) for bit in bitstring) % 2
        if parity == 0:
            exp_value += count
        else:
            exp_value -= count
    return exp_value / shots

def exp_ZiZj_from_counts(counts, i, j, n_qubits, shots):
    """
    Compute <Z_i Z_j> from Z-basis counts.
    Eigenvalue = +1 if bits the same, -1 if different.
    Qiskit bitstring order: left char = qubit N-1, right char = qubit 0.
    """
    total = 0
    for bitstring, c in counts.items():
        # map qubit index -> character index
        # qubit 0 corresponds to rightmost character
        bit_i = int(bitstring[n_qubits - 1 - i])
        print(f"bit_i for qubit {i}: {bit_i}")
        bit_j = int(bitstring[n_qubits - 1 - j])
        print(f"bit_j for qubit {j}: {bit_j}")
        eigenvalue = 1 if bit_i == bit_j else -1
        total += eigenvalue * c
    return total / shots

N = 6
E_XN = exp_XN_or_YN_from_counts(counts_X, N, SHOTS)
E_YN = exp_XN_or_YN_from_counts(counts_Y, N, SHOTS)
E_ZiZj = exp_ZiZj_from_counts(counts_Z, 0, 1, N, SHOTS)
